In [1]:
!apt install graphviz graphviz-dev
!pip install wget
!pip install pygraphviz

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'libgraphviz-dev' instead of 'graphviz-dev'
graphviz is already the newest version (2.40.1-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin libgtk2.0-common
  libgvc6-plugins-gtk libxdot4
Suggested packages:
  gvfs
The following NEW packages will be installed:
  libgail-common libgail18 libgraphviz-dev libgtk2.0-0 libgtk2.0-bin
  libgtk2.0-common libgvc6-plugins-gtk libxdot4
0 upgraded, 8 newly installed, 0 to remove and 34 not upgraded.
Need to get 2,120 kB of archives.
After this operation, 7,128 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libgtk2.0-common all 2.24.32-1ubuntu1 [125 kB]
Get:2 http://archive.ubuntu.com/ubuntu bio

In [2]:
import wget
import os

def download(target_name = 'iris.data'):
    """
    Download the mushroom dataset from UCI
    :param target_name: target path name
    """
    if not os.path.exists(target_name):
        url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
        wget.download(url, target_name)

download()

# Core Code

In [8]:
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, recall_score
import pygraphviz as pgv


class C45Classifier(object):

    def entropy(self,dataset,fea=-1):
        labels = dataset[:,fea]
        h = 0.0
        for k,v in zip(*np.unique(labels,return_counts=True)):
            h -= v / labels.shape[0] * np.log2([v / labels.shape[0]])[0]
        return h

    def bin_split_data(self,dataset,index,value):
        smaller = dataset[dataset[:,index]<=value]
        bigger = dataset[dataset[:,index]>value]
        return smaller, bigger

    def reg_loss(self,dataset):
        a = np.mean(dataset[:,-1])
        return np.square(dataset[:,-1]-a).sum()

    def info_gain(self, dataset, feature, val=None):
        if val == None:
            feature_values = np.unique(dataset[:,feature])
            entropy_sub_dataset = 0.0
            for val in feature_values:
                sub_dataset = dataset[dataset[:,feature]==val]
                entropy_sub_dataset += sub_dataset.shape[0]/dataset.shape[0] * self.entropy(sub_dataset)
            return self.entropy(dataset) - entropy_sub_dataset
        else:
            sm, bg = self.bin_split_data(dataset,feature,val)
            hda =- sm.shape[0]/dataset.shape[0]*self.entropy(sm) - bg.shape[0]/dataset.shape[0]*self.entropy(bg)
            hd = self.entropy(dataset)
            return hd-hda

    def info_gain_ratio(self, dataset, feature, val=None):
        if val == None:
            feature_values = np.unique(dataset[:,feature])
            entropy_sub_dataset = 0.0
            for val in feature_values:
                sub_dataset=dataset[dataset[:,feature]==val]
                entropy_sub_dataset += sub_dataset.shape[0]/dataset.shape[0] * self.entropy(sub_dataset)
            Had = self.entropy(dataset,feature)
            return (self.entropy(dataset) - entropy_sub_dataset)/Had
        else:
            sm, bg=self.bin_split_data(dataset,feature,val)
            hda =- sm.shape[0]/dataset.shape[0]*self.entropy(sm) - bg.shape[0]/dataset.shape[0]*self.entropy(bg)
            hd = self.entropy(dataset)
            had =- np.log2(sm.shape[0]/dataset.shape[0]) - np.log2(bg.shape[0]/dataset.shape[0])
            return (hd-hda)/had

    def select_feature(self, dataset, features1, features2):
        if dataset.shape[0]<=1*2:
            return None, self.majority_label(dataset[:,-1])

        if len(features1)!=0:
            info_gains = [(self.info_gain_ratio(dataset, x), x) for x in features1]
            loss = max(info_gains,key=lambda x:x[0])[0]
            best_index = max(info_gains,key=lambda x:x[0])[1]
        else:
            loss, best_index= -np.inf, 0
        best_split, uncon, c = 0, True, 0
  
        for i in features2:
            undata = np.unique(dataset[:,i])
            if undata.shape[0]==1:
                c += 1
            for val in undata:
                sm, bg = self.bin_split_data(dataset, i, val)
                if sm.shape[0]<1 or bg.shape[0]<1:
                    continue
                loss1 = self.info_gain_ratio(dataset,i,val)
                if loss<loss1:
                    loss, best_index, best_split, uncon = loss1, i, val, False
        if c == dataset.shape[1]-1:
            return None, self.majority_label(dataset[:,-1])
        if uncon:
            return max(info_gains)[1], None
        return best_index, best_split
     
    def majority_label(self, labels):
        return max(zip(*np.unique(labels,return_counts=True)),key=lambda x:x[1])[0]

    def build_tree(self, dataset, features1, features2):
        labels = dataset[:,-1]
        features = features1+features2
        if np.unique(labels).shape[0] == 1:
            return {'label': labels[0]}
        if len(features) == 0:
            return {'label': self.majority_label(labels)}
        best_feature, best_split = self.select_feature(dataset, features1, features2)
        if best_split == None:
            tree = {'feature': best_feature, 'children': {}}
            best_feature_values = np.unique(dataset[:, best_feature])
            for val in best_feature_values:
                sub_dataset = dataset[dataset[:, best_feature]==val]
                if len(sub_dataset) == 0:
                    tree['children'][val] = {
                        'label': self.majority_label(labels)}
                else:
                    tree['children'][val] = self.build_tree(
                        sub_dataset, [x for x in features1 if x != best_feature],features2)
        else:
            if best_feature == None:
                return {'label': best_split}
            tree = {'feature': best_feature, 'children': {},'best_split':best_split}
            sm, bg = self.bin_split_data(dataset, best_feature, best_split)
            tree['children']['left'] = self.build_tree(sm, features1, [x for x in features2 if x != best_feature])
            tree['children']['right'] = self.build_tree(bg, features1, [x for x in features2 if x != best_feature])
        return tree
        
    def fit(self, X, y):
        dataset = np.c_[X,y]
        feature1, feature2 = [], []
 
        for i in range(dataset.shape[1]-1):
            if np.unique(dataset[:,i]).shape[0]<=10:
                feature1.append(i)
            else:
                feature2.append(i)
        self.tree = self.build_tree(dataset, feature1, feature2)

    def _predict(self, tree, x):
        if 'feature' in tree:
            if 'best_split' in tree:
                if x[tree['feature']]<=tree['best_split']:
                    return self._predict(tree['children']['left'], x)
                else:
                    return self._predict(tree['children']['right'], x)
            else:
                return self._predict(tree['children'][x[tree['feature']]], x)
        else:
            return tree['label']

    def predict(self, X):
        return np.apply_along_axis(lambda x:self._predict(self.tree, x),1, X)

    def draw_tree(self, k):
        tr = pgv.AGraph()
        label_names = ["Iris-setosa", "Iris-versicolor", "Iris-virginica"]
        num = [0]
        def DFS(rt):
            if rt.get('children') != None:
                now = f"feature={rt['feature']}\nbest_split={rt['best_split']}"
                if rt['children'].get('left') != None:
                    ch = rt['children']['left']
                    Next = ""
                    if ch.get('feature') != None:
                        Next = f"feature={ch['feature']}\nbest_split={ch['best_split']}"
                    elif ch.get('label') != None:
                        tr.add_node(num[0], label=f"{label_names[int(ch['label'])]}")
                        Next = tr.get_node(num[0])
                        num[0] += 1
                  
                    tr.add_edge(now, Next)
                    DFS(ch)
                if rt['children'].get('right') != None:
                    ch = rt['children']['right']
                    Next = ""
                    if ch.get('feature') != None:
                        Next = f"feature={ch['feature']}\nbest_split={ch['best_split']}"
                    elif ch.get('label') != None:
                        tr.add_node(num[0], label=f"{label_names[int(ch['label'])]}")
                        Next = tr.get_node(num[0])
                        num[0] += 1
        
                    tr.add_edge(now, Next)
                    DFS(ch)
        DFS(self.tree)
        tr.layout("dot")
        tr.draw(f"file_{k}.png")
        
    def score(self, X, y, k):
         y_pred = self.predict(X)
         print(f'Accuracy = {accuracy_score(y, y_pred)}')
         print(f'Confusion Matrix= {confusion_matrix(y, y_pred)}')
         print(f"F1 Score = {f1_score(y, y_pred, average='weighted')}")
         print(f"Recall Score = {recall_score(y, y_pred, average='weighted')}")
         self.draw_tree(k)
    
         
         

## Data Preprocessing

In [9]:
dataset_path = 'iris.data'
X = np.delete(np.genfromtxt(dataset_path, delimiter=","), 4, 1)

y = []
label_dict = {
    "Iris-setosa": 0,
    "Iris-versicolor": 1,
    "Iris-virginica": 2
}
with open(dataset_path) as file:
    lines = file.read().splitlines()
    for line in lines:
        if not line == "": 
            y.append(label_dict[line.split(",")[-1]])


# Result

In [10]:
from sklearn.model_selection import train_test_split


DecisionTree = C45Classifier()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

DecisionTree.fit(X_train, y_train)
DecisionTree.score(X_test, y_test, 0)

Accuracy = 0.9111111111111111
Confusion Matrix= [[14  2  0]
 [ 1 17  0]
 [ 0  1 10]]
F1 Score = 0.9118459230513559
Recall Score = 0.9111111111111111


In [11]:
from sklearn.model_selection import KFold

i = 1
kf = KFold(n_splits=4, random_state=0, shuffle=True)
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = np.array(y)[train_index], np.array(y)[test_index]
    DecisionTree = C45Classifier()
    DecisionTree.fit(X_train, y_train)
    DecisionTree.score(X_test, y_test, i)
    i+=1


Accuracy = 0.8157894736842105
Confusion Matrix= [[11  2  0]
 [ 1 11  4]
 [ 0  0  9]]
F1 Score = 0.8142517736347137
Recall Score = 0.8157894736842105
Accuracy = 0.8421052631578947
Confusion Matrix= [[ 9  0  0]
 [ 3  8  3]
 [ 0  0 15]]
F1 Score = 0.8298017771701982
Recall Score = 0.8421052631578947
Accuracy = 0.918918918918919
Confusion Matrix= [[15  1  0]
 [ 0  5  2]
 [ 0  0 14]]
F1 Score = 0.9171662978114591
Recall Score = 0.918918918918919
Accuracy = 0.8108108108108109
Confusion Matrix= [[12  0  0]
 [ 3 10  0]
 [ 2  2  8]]
F1 Score = 0.8089468779123952
Recall Score = 0.8108108108108109
